# 导入数据包

In [1]:
# -*- coding: utf-8 -*-
# @File  : 中文文本分类20200609-4分类版本.ipynb
# @Author: 英俊
# @Date  : 2020/6/9
# @Software: jupyter notebook
# 复旦大学语料文本分类

import pandas as pd

import jieba

import os

import pickle  # 持久化

from numpy import *

#数据提取
import sklearn

from sklearn.feature_extraction.text import TfidfTransformer  # TF-IDF向量转换类

from sklearn.feature_extraction.text import TfidfVectorizer  # TF_IDF向量生成类


#模型建立
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.naive_bayes import BernoulliNB,MultinomialNB,GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import xgboost
from xgboost import XGBClassifier
# 集成模块
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,AdaBoostRegressor
from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,AdaBoostRegressor

#Pipeline 使用一系列 (key, value) 键值对来构建,其中 key 是你给这个步骤起的名字， value 是一个评估器对象:
from sklearn.pipeline import Pipeline
#准确率，精确率，召回率，f1
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,classification_report
# 保存数据
import joblib

# 读取数据
def readFile(path):
    with open(path, 'r', errors='ignore') as file:  # 文档中编码有些问题，所有用errors过滤错误
        content = file.read()
        return content

# 这块没用，供日后研究一下
# 保存文件
def saveFile(path, result):
    with open(path, 'w', errors='ignore') as file:
        file.write(result)

In [2]:
import jieba

import os

import pickle  # 持久化

from numpy import *

In [3]:
def get_data():
    resultList=[]
    labeList=[]
    inputPath="./data/"
    fatherLists = os.listdir(inputPath)  # 主目录
    for eachDir in fatherLists:  # 遍历主目录中各个文件夹
        print(eachDir)
        eachPath = inputPath + eachDir + "/"  # 保存主目录中每个文件夹目录，便于遍历二级文件
        print(eachPath)
        childLists = os.listdir(eachPath)  # 获取每个文件夹中的各个文件
    #     print(eachPath)
    #     each_resultPath = resultPath + eachDir + "/"  # 分词结果文件存入的目录
        for eachFile in childLists:  # 遍历每个文件夹中的子文件
                    eachPathFile = eachPath + eachFile  # 获得每个文件路径
                    content = readFile(eachPathFile)  # 调用上面函数读取内容
                    result = (str(content)).replace("\r\n", "").strip()  # 删除多余空行与空格
                    result = content.replace("\r\n","").strip()
                    labeList.append(eachDir)
                    resultList.append(result)
                    cutResult = jieba.cut(result)  # 默认方式分词，分词结果用空格隔开
    return labeList,resultList

# 查看获取的标签与文本是否完整

In [4]:
labeList,resultList=get_data()

C3-Art
./data/C3-Art/
C4-Literature
./data/C4-Literature/
C5-Education
./data/C5-Education/
C6-Philosophy
./data/C6-Philosophy/


In [5]:
print(len(labeList))
print(len(resultList))

825
825


## 创建pandas对象进行查看

In [6]:
dictTmp={ "label":labeList ,"result":resultList}
df_all=pd.DataFrame(dictTmp)
df_all.head()

,label,result
0,C3-Art,【 文献号 】1-2340\n【原文出处】中国图书评论\n【原刊地名】沈阳\n【原刊期号】1...
1,C3-Art,【 文献号 】1-2681\n【原文出处】武汉大学学报：哲社版\n【原刊期号】199605\...
2,C3-Art,【 文献号 】2-4116\n【原文出处】云南学术探索\n【原刊地名】昆明\n【原刊期号】1...
3,C3-Art,【 文献号 】1-3036\n【原文出处】文艺理论研究\n【原刊地名】沪\n【原刊期号】19...
4,C3-Art,【 文献号 】1-3066\n【原文出处】文学评论\n【原刊地名】京\n【原刊期号】1995...


## 查看数据类型

In [7]:
df_all["result"][0]

'【 文献号 】1-2340\n【原文出处】中国图书评论\n【原刊地名】沈阳\n【原刊期号】199510\n【原刊页号】61-62\n【分 类 号】Z1\n【分 类 名】出版工作、图书评介\n【 作  者 】杨小民\n【复印期号】199602\n【 标  题 】图书评论应当重视对书籍装帧艺术的评价\n【 正  文 】\n    图书评论是近代报刊业兴起后，在世界各国得到长足发展的一种新型评论体裁。而不论是书评理论还是书评实践都有一个不小的疏漏，即忽\n视了图书的形式因素。因为图书是内容与形式的综合体，忽视了“图书形式”这一重要方面，会导致在图书评论活动中忽视对图书的出版形式这\n一重要方面的品评论述，而这对于出版物的达到基本要求：“形神俱佳”（“形”指书装艺术，“神”指内容叙述）或最高要求“尽善尽美”（\n“尽善”指内容而言，“尽美”指形式而言）无疑是有缺憾的。\n    图书的形式因素即为书籍的装帧设计艺术（以下简称“书装艺术”）。它的内容应当包括：封面、封底、书脊、环衬、扉页、字体、字号、\n插图、版式、护封等。装帧设计应是图书中的重要内容，顺理成章地应成为书评文章中不可或缺的评论对象。然而，在当前报刊上大量刊登的书\n评文章中谈及这一方面的极为少见。这一偏颇势必会对中国出版物综合水平的提高产生不良的影响。\n    图书出版事业是人类的思维活动和精神成果与科学技术相结合的一项系统工程。而书装艺术则渗透着“出版人”的思维活动和印刷科技的水\n平两个因素。设计者的艺术构思，通过印刷工艺的精心制作，与图书的内容达到协调一致，才形成一本精美的形神俱佳的图书。\n    如今，我国的一些出版社，对图书的装帧设计重视不够，这既成为书评作者忽视书装艺术的评论的一个潜因，他们认为许多图书的书装艺术\n不值一提或难以一说；同时，也人为地造成了对书装艺术粗糙现象的不合理宽容。究其原因，出版社不愿投入应有的资金和人力是主要问题。书\n装艺术本身也是体现出版物品位高低的一项重要因素。在现代图书出版印刷中，应投入必要的资金，以避免参加国际图书博览会的中国图书再被\n人们讥笑为“展翅高飞”、“鞠躬尽瘁”了。（由于纸质差，装订落后，我国图书陈列于国际展台时，暖气会使书册张开弯曲，这叫“展翅高飞\n”；还有则为书脊软塌，不能直立，弯腰驼背，则称“鞠躬尽瘁”。）\n    编辑素养的欠缺，也直

# 数据清洗
（根据不同的文本对象进行不同的处理，不要轻易加数据）

In [8]:
def remove_text(text):
    text=text.replace('\n','')
    text=text.replace(' ','')
    text=text.replace('【','')
    text=text.replace('】','')
    text=text.replace('（','')
    text=text.replace('）','')
    text=text.replace('、','')
    text=text.replace('。','')
    text=text.replace('；','')
    text=text.replace('“','')
    text=text.replace('”','')
    text=text.replace('…','')
    text=text.replace('，','')
    text=text.replace('：','')
    text=text.replace('《','')
    text=text.replace('》','')
    text=text.replace('[','')
    text=text.replace(']','')
    text=text.replace('—','')
    return text

## 查看清洗后的结果

In [9]:
txt=remove_text(df_all["result"][0])
print(txt)

文献号1-2340原文出处中国图书评论原刊地名沈阳原刊期号199510原刊页号61-62分类号Z1分类名出版工作图书评介作者杨小民复印期号199602标题图书评论应当重视对书籍装帧艺术的评价正文图书评论是近代报刊业兴起后在世界各国得到长足发展的一种新型评论体裁而不论是书评理论还是书评实践都有一个不小的疏漏即忽视了图书的形式因素因为图书是内容与形式的综合体忽视了图书形式这一重要方面会导致在图书评论活动中忽视对图书的出版形式这一重要方面的品评论述而这对于出版物的达到基本要求形神俱佳形指书装艺术神指内容叙述或最高要求尽善尽美尽善指内容而言尽美指形式而言无疑是有缺憾的图书的形式因素即为书籍的装帧设计艺术以下简称书装艺术它的内容应当包括封面封底书脊环衬扉页字体字号插图版式护封等装帧设计应是图书中的重要内容顺理成章地应成为书评文章中不可或缺的评论对象然而在当前报刊上大量刊登的书评文章中谈及这一方面的极为少见这一偏颇势必会对中国出版物综合水平的提高产生不良的影响图书出版事业是人类的思维活动和精神成果与科学技术相结合的一项系统工程而书装艺术则渗透着出版人的思维活动和印刷科技的水平两个因素设计者的艺术构思通过印刷工艺的精心制作与图书的内容达到协调一致才形成一本精美的形神俱佳的图书如今我国的一些出版社对图书的装帧设计重视不够这既成为书评作者忽视书装艺术的评论的一个潜因他们认为许多图书的书装艺术不值一提或难以一说同时也人为地造成了对书装艺术粗糙现象的不合理宽容究其原因出版社不愿投入应有的资金和人力是主要问题书装艺术本身也是体现出版物品位高低的一项重要因素在现代图书出版印刷中应投入必要的资金以避免参加国际图书博览会的中国图书再被人们讥笑为展翅高飞鞠躬尽瘁了由于纸质差装订落后我国图书陈列于国际展台时暖气会使书册张开弯曲这叫展翅高飞还有则为书脊软塌不能直立弯腰驼背则称鞠躬尽瘁编辑素养的欠缺也直接影响到书装艺术的优劣在我国的出版业中编辑通常是提供书装要求并参与设计方案的参与的前提应该是要具备一定的艺术素质和审美眼光但如今有相当一部分编辑缺乏这一点他们对艺术规律对美术设计者从事的工作特性知之甚少他们的参与从某种意义上来说甚至成为一种盲目的干涉外行指挥内行大至约束个框子小至书名作者的位置安放和颜色的指派不难设想在这种缺乏平等探讨的格局下要求所设计出来的封扉等的艺术效果将是什么样子当然提出这些问题并不

# 数据清洗

In [10]:
df_all['result_pred'] =[remove_text(i) for i in df_all['result']]

In [11]:
df_all.head()

,label,result,result_pred
0,C3-Art,【 文献号 】1-2340\n【原文出处】中国图书评论\n【原刊地名】沈阳\n【原刊期号】1...,文献号1-2340原文出处中国图书评论原刊地名沈阳原刊期号199510原刊页号61-62分类...
1,C3-Art,【 文献号 】1-2681\n【原文出处】武汉大学学报：哲社版\n【原刊期号】199605\...,文献号1-2681原文出处武汉大学学报哲社版原刊期号199605原刊页号136-137分类号...
2,C3-Art,【 文献号 】2-4116\n【原文出处】云南学术探索\n【原刊地名】昆明\n【原刊期号】1...,文献号2-4116原文出处云南学术探索原刊地名昆明原刊期号199706原刊页号77～81分类...
3,C3-Art,【 文献号 】1-3036\n【原文出处】文艺理论研究\n【原刊地名】沪\n【原刊期号】19...,文献号1-3036原文出处文艺理论研究原刊地名沪原刊期号199504原刊页号6-17分类号J...
4,C3-Art,【 文献号 】1-3066\n【原文出处】文学评论\n【原刊地名】京\n【原刊期号】1995...,文献号1-3066原文出处文学评论原刊地名京原刊期号199506原刊页号91-102分类号J...


 Label标签化

In [12]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
x=[]
encoder_x=LabelEncoder()
x=encoder_x.fit_transform(df_all['label'])
# print(lelist)
df_all['label_pred']=x
df_all.head()

,label,result,result_pred,label_pred
0,C3-Art,【 文献号 】1-2340\n【原文出处】中国图书评论\n【原刊地名】沈阳\n【原刊期号】1...,文献号1-2340原文出处中国图书评论原刊地名沈阳原刊期号199510原刊页号61-62分类...,0
1,C3-Art,【 文献号 】1-2681\n【原文出处】武汉大学学报：哲社版\n【原刊期号】199605\...,文献号1-2681原文出处武汉大学学报哲社版原刊期号199605原刊页号136-137分类号...,0
2,C3-Art,【 文献号 】2-4116\n【原文出处】云南学术探索\n【原刊地名】昆明\n【原刊期号】1...,文献号2-4116原文出处云南学术探索原刊地名昆明原刊期号199706原刊页号77～81分类...,0
3,C3-Art,【 文献号 】1-3036\n【原文出处】文艺理论研究\n【原刊地名】沪\n【原刊期号】19...,文献号1-3036原文出处文艺理论研究原刊地名沪原刊期号199504原刊页号6-17分类号J...,0
4,C3-Art,【 文献号 】1-3066\n【原文出处】文学评论\n【原刊地名】京\n【原刊期号】1995...,文献号1-3066原文出处文学评论原刊地名京原刊期号199506原刊页号91-102分类号J...,0


查看数据的不平衡性，以后要进行修改

In [13]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
x=[]
encoder_x=LabelEncoder()
x=encoder_x.fit_transform(df_all['label'])
# onehotencoder = onehotencoder(categorical_features = [0])
# x=onehotencoder.fit_transform(x).toarray()
# x=[:,1:]
print(x)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

## 进行停用词清洗

In [14]:
# -*- encoding:utf-8 -*-
import codecs

import jieba

import jieba.analyse

import jieba.posseg

import pandas as pd

# import param

# import util

# import param

# import util

stopwords = {}
for line in codecs.open('stopword.txt', 'r', 'utf-8'):
    stopwords[line.rstrip()] = 1
############################ 定义分词函数 ############################

def split_word(text, stopwords):
    word_list = jieba.cut(text)
    start = True
    result = ''
    for word in word_list:
        word = word.strip()
        if word not in stopwords:
            if start:
                result = word
                start = False
            else:
                result += word
    return result

这个步骤最耗时

In [15]:
# pd_all['review_pred'] =[text_remove_punctuation(i) for i in pd_all['review']]
df_all['result_pred_test'] =[split_word(i,stopwords) for i in df_all['result_pred']]
df_all.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.450 seconds.
Prefix dict has been built successfully.


,label,result,result_pred,label_pred,result_pred_test
0,C3-Art,【 文献号 】1-2340\n【原文出处】中国图书评论\n【原刊地名】沈阳\n【原刊期号】1...,文献号1-2340原文出处中国图书评论原刊地名沈阳原刊期号199510原刊页号61-62分类...,0,文献号2340原文出处图书评论原刊地名原刊期号199510原刊页号6162分类号Z1分类名图...
1,C3-Art,【 文献号 】1-2681\n【原文出处】武汉大学学报：哲社版\n【原刊期号】199605\...,文献号1-2681原文出处武汉大学学报哲社版原刊期号199605原刊页号136-137分类号...,0,文献号2681原文出处武汉大学学报哲社版原刊期号199605原刊页号136137分类号Z1分...
2,C3-Art,【 文献号 】2-4116\n【原文出处】云南学术探索\n【原刊地名】昆明\n【原刊期号】1...,文献号2-4116原文出处云南学术探索原刊地名昆明原刊期号199706原刊页号77～81分类...,0,文献号4116原文出处学术原刊地名原刊期号199706原刊页号7781分类号J6分类名音乐舞...
3,C3-Art,【 文献号 】1-3036\n【原文出处】文艺理论研究\n【原刊地名】沪\n【原刊期号】19...,文献号1-3036原文出处文艺理论研究原刊地名沪原刊期号199504原刊页号6-17分类号J...,0,文献号3036原文出处文艺理论原刊地名沪原刊期号199504原刊页号17分类号J1分类名文艺...
4,C3-Art,【 文献号 】1-3066\n【原文出处】文学评论\n【原刊地名】京\n【原刊期号】1995...,文献号1-3066原文出处文学评论原刊地名京原刊期号199506原刊页号91-102分类号J...,0,文献号3066原文出处文学评论原刊地名京原刊期号199506原刊页号91102分类号J1分类...


In [16]:
#将数据随机分为3:1
feature=df_all['result_pred_test']
target=df_all['label_pred']
X_train, X_test, y_train, y_test = train_test_split(feature,target,test_size=0.3, random_state=33)

In [17]:
# 数据查看
import numpy as np
import pandas as pd
import sklearn
#机器学习算法模型
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.naive_bayes import BernoulliNB,MultinomialNB,GaussianNB
from sklearn.neighbors import KNeighborsClassifier
# 特征提取
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
#Pipeline 使用一系列 (key, value) 键值对来构建,其中 key 是你给这个步骤起的名字， value 是一个评估器对象:
from sklearn.pipeline import Pipeline
#准确率，精确率，召回率，f1
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,classification_report
import joblib

In [18]:
# 创建各类cv=CountVectorizer()和tf_idf工具
cv=CountVectorizer()
tdf=TfidfVectorizer()

In [19]:
#%%
# Count vectoriser --> LogisticRegression()

# 分类模型

#1.逻辑回归
lr=LogisticRegression()
# 贝叶斯
#2.多项式贝叶斯
mb=MultinomialNB()
#3.伯努利贝叶斯
bb=BernoulliNB()
# 支持向量机
#4.支持向量机
svc=SVC(kernel='rbf')
svc1=SVC(kernel='linear')
svc2=SVC(kernel='poly')
svc3=SVC(kernel='sigmoid')

#5.
linearsvc=LinearSVC()
#6.决策树
dtc=DecisionTreeClassifier(random_state=22)
#7.随机森林
rfc=RandomForestClassifier(random_state=22)
#9.KNN分类器
knn=KNeighborsClassifier()

modelList=[lr,mb,bb,svc,svc1,svc2,svc3,linearsvc,dtc,rfc,knn]

#11个模型
m_len=len(modelList)

# 形成 9个模型 2个提取特征 5个指标  
# 提取特征分类器
textVectoriser=[cv,tdf]
textv_len=len(textVectoriser)


In [20]:
accuracy_score_list=[]
precision_score_list=[]
recall_score_list=[]
f1_score_list=[]
classification_report_list=[]
modelClass=[]

In [21]:
for i in range(m_len):
    for j in range(textv_len):
        pipeline = Pipeline([('vectorizer', textVectoriser[j]), ('classifier', modelList[i])])   
        pipeline.fit(X_train,y_train)
        pred=pipeline.predict(X_test)
        accuracy_score_list.append(accuracy_score(y_test,pred))
        #以为以下的有些只适合二分类
#         precision_score_list.append(precision_score(y_test,pred))
#         f1_score_list.append(f1_score(y_test,pred))
#         recall_score_list.append(f1_score(y_test,pred))
# #         classification_report_list.append(classification_report(y_test,pred))
#         modelClass.append(pipeline)    

In [22]:
new_ticks = []
name=[]
modelNamelist=['逻辑回归','多项式贝叶斯','伯努利贝叶斯','RBF核SVM'
               ,'线性核SVM','多项式SVM','sigmoidSVM'
               ,'LinearSVC','决策树','随机森林','KNN']
modelNamelist2=['lr','mb','bb','svc','svc1','svc2','svc3','l'+'\n'+'svc','dtc','rfc','knn']
textVectorNamelist = ['词袋','TDF']
for i in range(m_len):
    for j in range(textv_len):
         new_ticks.append('\n'.join([modelNamelist2[i],textVectorNamelist[j]]))
         name.append(modelNamelist[i]+textVectorNamelist[j])
         

In [23]:
import matplotlib.pyplot as plt
from pylab import *
mpl.rcParams['font.sans-serif'] = ['SimHei']
#plot根据列表绘制出有意义的图形，linewidth是图形线宽，可省略
# plt.plot(input_values,squares,linewidth=5)
plt.figure(figsize=(12,5),dpi=80)
print(len(accuracy_score_list))
plt.bar(range(len(accuracy_score_list)),accuracy_score_list,linewidth=5)
#设置图标标题
plt.title("不同管道模型准确率",fontsize = 24)
#设置坐标轴标签
plt.xlabel("模型类型",fontsize = 0.2)
plt.ylabel("准确率",fontsize = 0.5)
#设置刻度标记的大小
plt.tick_params(axis='both',labelsize = 14)
#打开matplotlib查看器，并显示绘制图形
#这是一半
plt.xticks(range(22),new_ticks[:22])
plt.show()
# plt.xticks(new_ticks)
# plt.xticks(new)


# print(new_ticks[max_index_index])

22


In [24]:
max_value=max(accuracy_score_list)
index = accuracy_score_list.index(max_value)
print('准确率最高的是',name[index])
print('max',max_value)
final_index=index
final_name=name[index]

准确率最高的是 线性核SVM词袋
max 0.8467741935483871
